In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf


from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


In [2]:
#  Import and read the charity_data.csv.
import pandas as pd 
motor_df = pd.read_csv('Resources/Motor_Vehicle_Crashes_-_Vehicle_Information__Three_Year_Window.csv')
motor_df.head()

,Year,Case Vehicle ID,Vehicle Body Type,Registration Class,Action Prior to Accident,Type / Axles of Truck or Bus,Direction of Travel,Fuel Type,Vehicle Year,State of Registration,Number of Occupants,Engine Cylinders,Vehicle Make,Contributing Factor 1,Contributing Factor 1 Description,Contributing Factor 2,Contributing Factor 2 Description,Event Type,Partial VIN
0,2018,15232120,VAN TRUCK,AGRICULTURAL TRUCK,Parked,Not Entered,North,Gas,2015.0,NY,0.0,4.0,NISSA,HUMAN,Not Entered,HUMAN,Not Entered,Not Entered,3N6CM0KN2FK713460
1,2018,15232535,SUBURBAN,PASSENGER OR SUBURBAN,Making U Turn,Not Entered,South,Gas,2015.0,NY,1.0,6.0,HONDA,HUMAN,Not Entered,HUMAN,Unknown,Not Entered,5J6TF2H55FL001556
2,2018,15232536,SEDAN,Not Entered,Going Straight Ahead,Not Entered,North,Not Entered,NaN,PA,1.0,NaN,NaN,HUMAN,Not Entered,HUMAN,Unknown,Not Entered,NaN
3,2018,15232557,4 DOOR SEDAN,PASSENGER OR SUBURBAN,Parked,Not Entered,South,Gas,2013.0,NY,5.0,4.0,NISSA,HUMAN,Passenger Distraction,VEHICLE,Other*,Not Entered,1N4AL3AP8DC229156
4,2018,15232556,SUBURBAN,PASSENGER OR SUBURBAN,Going Straight Ahead,Not Entered,South,Gas,2009.0,NY,2.0,8.0,CADIL,HUMAN,Not Applicable,HUMAN,Not Applicable,Not Entered,3GYFK22209G213337


# Cleaning the Data

In [3]:
# Drop all NA from rows

motor_df = motor_df.dropna()

In [4]:
# Get Names for indexs for which Contributing Factor  1 Description 1 is Not Entered
# Drop these rows

not_entered = motor_df[motor_df['Contributing Factor 1 Description'] == 'Not Entered'].index
motor_df.drop(not_entered, inplace = True)

In [5]:
# Get Names for indexs for which Contributing Factor  1 Description 1 is Not applicable
# Drop these rows

not_app = motor_df[motor_df['Contributing Factor 1 Description'] == 'Not Applicable'].index
motor_df.drop(not_app, inplace = True)

In [6]:
# Get Names for index for which Contributing Factor 1 Description is Unknown
# Drop these rows

unknown = motor_df[motor_df['Contributing Factor 1 Description'] == 'Unknown'].index
motor_df.drop(unknown, inplace = True)

In [7]:
# Get unknown values in direction of travel
# Drop these rows

unknown2 = motor_df[motor_df['Direction of Travel'] == 'Unknown'].index
motor_df.drop(unknown2, inplace = True)

In [8]:
# Get Not Applicable in Direction of Travel and drop these rows

not_app = motor_df[motor_df['Direction of Travel'] == 'Not Applicable'].index
motor_df.drop(not_app, inplace = True)

In [9]:
# Looking at the Vehicle Make column
motor_count = motor_df['Vehicle Make'].value_counts()

# Determine which values to replace if counts are less than 5000
replace_Vehicle = list(motor_count[motor_count < 3000].index)
replace_Vehicle

# Replace in dataframe
for car in replace_Vehicle:
    motor_df["Vehicle Make"] = motor_df["Vehicle Make"].replace(car,"Other")
    
# Check to make sure binning was successful
motor_df['Vehicle Make'].value_counts()

TOYOT    57239
HONDA    46095
CHEVR    44133
FORD     44126
NISSA    35334
Other    33098
JEEP     21089
HYUND    18128
DODGE    16350
SUBAR    13317
KIA       9343
GMC       8263
BMW       8175
ME/BE     7529
VOLKS     7428
CHRYS     7369
LEXUS     5752
ACURA     5745
MAZDA     5685
RAM       4640
BUICK     4562
INFIN     4461
MITSU     3586
AUDI      3495
CADIL     3133
Name: Vehicle Make, dtype: int64

In [10]:
# Recode Vehicle Make to integer: LinearRegression does not take string

def Classification(X):
    if X == 'TOYOT':
        return 0
    elif X == 'HONDA':
        return 1
    elif X == 'CHEVR':
        return 2
    elif X == 'FORD':
        return 3
    elif X == 'NISSA':
        return 4
    elif X == 'Other':
        return 5
    elif X == 'JEEP':
        return 6
    elif X == 'HYUND':
        return 7
    elif X == 'DODGE':
        return 8
    elif X == 'SUBAR':
        return 9
    elif X == 'KIA':
        return 10
    elif X == 'GMC':
        return 11
    elif X == 'BMW':
        return 12
    elif X == 'ME/BE':
        return 13
    elif X == 'VOLKS':
        return 14
    elif X == 'CHRYS':
        return 15
    elif X == 'LEXUS':
        return 16
    elif X == 'ACURA':
        return 17
    elif X == 'MAZDA':
        return 18
    elif X == 'RAM':
        return 19
    elif X == 'BUICK':
        return 20
    elif X == 'INFIN':
        return 21
    elif X == 'MITSU':
        return 22
    elif X == 'AUDI':
        return 23
    else :
        return 24

motor_df['Vehicle Make'] = motor_df['Vehicle Make'].apply(Classification)

In [11]:
# Look at Regristration Class value counts for binning
class_count = motor_df['Registration Class'].value_counts()


# Determine which values to replace if counts are less than 4000
replace_class = list(class_count[class_count < 2000].index)

# Replace in dataframe
for Class in replace_class:
    motor_df['Registration Class'] = motor_df['Registration Class'].replace(Class,"Other")
    
# Check to make sure binning was successful
motor_df['Registration Class'].value_counts()

PASSENGER OR SUBURBAN         347712
OMNIBUS - TAXI                 20924
AGRICULTURAL TRUCK             14251
Other                          10599
COMMERCIAL                      9818
SPECIAL PASSENGER               5611
POLITICAL SUBDIVISION           4352
MOTORCYCLE                      2771
INTERNATIONAL REGISTRATION      2037
Name: Registration Class, dtype: int64

In [12]:
# Action Prior to accident
actions = motor_df['Action Prior to Accident'].value_counts()

# Determine which values to replace if counts are less than 4000
replace_action = list(actions[actions < 2000].index)

# Replace in dataframe
for action in replace_action:
    motor_df['Action Prior to Accident'] = motor_df['Action Prior to Accident'].replace(action,"Other")
    
# Check to make sure binning was successful
motor_df['Action Prior to Accident'].value_counts()

Going Straight Ahead        243029
Making Left Turn             46153
Backing                      23642
Slowing or Stopping          22816
Making Right Turn            21186
Changing Lanes               20355
Starting in Traffic           7221
Merging                       5984
Other                         5926
Starting from Parking         5644
Overtaking/Passing            4366
Making U Turn                 4236
Entering Parked Position      3154
Stopped in Traffic            2309
Parked                        2054
Name: Action Prior to Accident, dtype: int64

In [13]:
# Look at Vehicle Body Type value counts for binning
body_types = motor_df['Vehicle Body Type'].value_counts()


# Determine which values to replace if counts are less than 4000
replace_body = list(body_types[body_types < 1000].index)

# Replace in dataframe
for body_type in replace_body:
    motor_df['Vehicle Body Type'] = motor_df['Vehicle Body Type'].replace(body_type,"Other")
    
# Check to make sure binning was successful
motor_df['Vehicle Body Type'].value_counts()

SUBURBAN          168435
4 DOOR SEDAN      168355
PICKUP TRUCK       35355
2 DOOR SEDAN       13394
VAN TRUCK           9017
TAXI                3877
BUS (OMNIBUS)       3346
Other               3202
DUMP                2962
MOTORCYCLE          2812
CONVERTIBLE         2332
TRACTOR             2059
DELIVERY TRUCK      1790
UTILITY             1139
Name: Vehicle Body Type, dtype: int64

In [14]:
def Classification(X):
    if X == 'SUBURBAN':
        return 0
    elif X == '4 DOOR SEDAN':
        return 1
    elif X == 'PICKUP TRUCK':
        return 2
    elif X == '2 DOOR SEDAN':
        return 3
    elif X == 'VAN TRUCK':
        return 4
    elif X == 'TAXI':
        return 5
    elif X == 'BUS (OMNIBUS)':
        return 6
    elif X == 'Other':
        return 7
    elif X == 'DUMP':
        return 8
    elif X == 'MOTORCYCLE':
        return 9
    elif X == 'CONVERTIBLE':
        return 10
    elif X == 'TRACTOR':
        return 11
    elif X == 'DELIVERY TRUCK':
        return 12
    else:
        return 13
    
motor_df['Vehicle Body Type'] = motor_df['Vehicle Body Type'].apply(Classification)    

In [15]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME','Contributing Factor 1 Description'
# and 'Contributing Factor 2 Description'.

motor_df = motor_df.drop(['Case Vehicle ID', 'Partial VIN', 'Contributing Factor 2','Contributing Factor 2 Description'], axis=1)

# Creating target and Features for our model

In [16]:
# Create our target
y = motor_df['Vehicle Make'] 

# Create our features
X = motor_df.drop(columns = 'Vehicle Make')
X = pd.get_dummies(X)

In [17]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [18]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, criterion = 'entropy', random_state=78) 

Training the model

In [20]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [21]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [22]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Transforming the confusion matrix to dataframe for easy readability.
cm_df = pd.DataFrame(
    cm, index=["TOYOTA", "HONDA", "CHEVR", "FORD", "NISSAN", "Other", "JEEP", "HYUNDAI", "DODGE",
               "SUBARU", "KIA", "GMC", "BMW", "MERCEDES", "VOLKSWAGEN", "CHRYSLER", "LEXUS",
               "ACURA", "MAZDA", "RAM", "BUICK", "INFINIY", "MITSUBISHI", "AUDI", "CADILLAC"])

cm_df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
TOYOTA,5084,2263,1115,864,1242,307,752,545,314,382,...,147,92,113,102,65,71,86,58,50,20
HONDA,2515,2739,859,644,1201,373,681,561,208,537,...,118,76,86,109,11,68,66,74,47,19
CHEVR,1148,1088,2626,1798,755,359,495,355,441,309,...,108,110,80,57,283,108,78,27,35,54
FORD,1240,930,1821,3248,667,426,677,225,402,221,...,79,84,58,43,179,59,52,44,26,49
NISSAN,1850,1545,827,686,1203,211,570,413,150,363,...,83,90,75,80,26,59,69,31,43,27
Other,610,629,529,574,273,4371,287,117,121,80,...,50,84,40,25,9,74,44,32,23,60
JEEP,568,501,423,534,443,115,1422,65,225,285,...,103,65,62,37,6,34,35,24,13,26
HYUNDAI,949,983,438,271,530,111,159,354,69,137,...,32,16,32,36,1,28,18,19,19,12
DODGE,449,335,659,634,233,146,440,86,573,31,...,75,50,44,14,6,18,44,4,12,24
SUBARU,459,690,300,201,385,41,205,117,23,543,...,26,2,8,41,0,8,2,22,11,3


In [23]:
# Creating a table (dataframe for classification report

report = classification_report(y_test, predictions, output_dict=True)
report_df = pd.DataFrame(report).transpose()
modified_report_df = report_df.rename(index = {"0" : "TOYOTA", "1": "HONDA", "2": "CHEVR", "3": "FORD", "4":"NISSAN", 
                      "5": "Other", "6": "JEEP", "7": "HYUNDAI", "8":"DODGE","9":"SUBARU", "10": "KIA", "11": "GMC", 
                      "12": "BMW", "13": "MERCEDES", "14": "VOLKSWAGEN", "15": "CHRYSLER", "16": "LEXUS", "17":"ACURA", 
                      "18": "MAZDA", "19": "RAM", "20": "BUICK", "21": "INFINIY", "22": "MITSUBISHI", "23": "AUDI", "24":"CADILLAC" })


In [25]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [26]:
# Displaying results

print(f"Accuracy : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))
modified_report_df.round(2)

Accuracy : 0.22869526114869068
Classification Report
              precision    recall  f1-score   support

           0       0.29      0.36      0.32     14231
           1       0.19      0.24      0.21     11568
           2       0.21      0.24      0.22     10981
           3       0.28      0.29      0.29     11071
           4       0.14      0.14      0.14      8906
           5       0.57      0.52      0.54      8387
           6       0.19      0.27      0.22      5241
           7       0.10      0.08      0.09      4428
           8       0.18      0.14      0.16      4128
           9       0.15      0.17      0.16      3218
          10       0.05      0.03      0.03      2270
          11       0.07      0.04      0.05      2094
          12       0.10      0.06      0.07      2101
          13       0.09      0.06      0.07      1847
          14       0.40      0.26      0.32      1835
          15       0.08      0.05      0.06      1944
          16       0.06     

,precision,recall,f1-score,support
TOYOTA,0.29,0.36,0.32,14231.00
HONDA,0.19,0.24,0.21,11568.00
CHEVR,0.21,0.24,0.22,10981.00
FORD,0.28,0.29,0.29,11071.00
NISSAN,0.14,0.14,0.14,8906.00
Other,0.57,0.52,0.54,8387.00
JEEP,0.19,0.27,0.22,5241.00
HYUNDAI,0.10,0.08,0.09,4428.00
DODGE,0.18,0.14,0.16,4128.00
SUBARU,0.15,0.17,0.16,3218.00
